In [3]:
import numpy as np

# -------------------------------
# Problem setup
# -------------------------------
np.random.seed(42)  # for repeatability

# Electricity price for each of 24 hours ($/kWh)
prices = np.random.uniform(0.1, 0.4, 24)  # random daily price pattern

# List of appliances: (name, power usage in kW, duration in hours)
appliances = [
    ("Washing Machine", 1.0, 2),
    ("Dishwasher", 1.2, 2),
    ("Heater", 2.5, 3),
    ("Air Conditioner", 3.0, 4)
]

num_appliances = len(appliances)
hours = len(prices)

# -------------------------------
# GWO Parameters
# -------------------------------
num_wolves = 20   # number of candidate schedules
max_iter = 100    # number of optimization rounds

# Each wolf: an array of start times for each appliance (0–23)
positions = np.random.randint(0, hours, (num_wolves, num_appliances))

# -------------------------------
# Fitness function (what we minimize)
# -------------------------------
def fitness(schedule):
    """Calculate total cost of a given schedule."""
    total_cost = 0
    for i, (_, power, duration) in enumerate(appliances):
        start = int(schedule[i])
        for h in range(start, start + duration):
            total_cost += power * prices[h % hours]  # wrap around at 24 hours
    return total_cost

# -------------------------------
# Initialize alpha, beta, delta wolves (best three)
# -------------------------------
alpha, beta, delta = None, None, None
alpha_score = beta_score = delta_score = float("inf")

# -------------------------------
# Main Grey Wolf Optimization loop
# -------------------------------
for t in range(max_iter):
    # Evaluate fitness of each wolf
    for i in range(num_wolves):
        cost = fitness(positions[i])
        if cost < alpha_score:
            alpha_score, alpha = cost, positions[i].copy()
        elif cost < beta_score:
            beta_score, beta = cost, positions[i].copy()
        elif cost < delta_score:
            delta_score, delta = cost, positions[i].copy()
    
    # a decreases linearly from 2 to 0
    a = 2 - t * (2 / max_iter)
    
    # Update each wolf's position
    for i in range(num_wolves):
        for j in range(num_appliances):
            # Random coefficients
            r1, r2 = np.random.rand(), np.random.rand()
            A1, C1 = 2 * a * r1 - a, 2 * r2
            D_alpha = abs(C1 * alpha[j] - positions[i][j])
            X1 = alpha[j] - A1 * D_alpha

            r1, r2 = np.random.rand(), np.random.rand()
            A2, C2 = 2 * a * r1 - a, 2 * r2
            D_beta = abs(C2 * beta[j] - positions[i][j])
            X2 = beta[j] - A2 * D_beta

            r1, r2 = np.random.rand(), np.random.rand()
            A3, C3 = 2 * a * r1 - a, 2 * r2
            D_delta = abs(C3 * delta[j] - positions[i][j])
            X3 = delta[j] - A3 * D_delta

            # New average position
            new_pos = (X1 + X2 + X3) / 3

            # Make sure it's within valid time range
            positions[i][j] = np.clip(new_pos, 0, hours - 1)

    # Optional: show progress every 20 iterations
    if (t + 1) % 20 == 0:
        print(f"Iteration {t+1:3d} | Best cost so far: ${alpha_score:.2f}")

# -------------------------------
# Final Results
# -------------------------------
print("\n=== Optimal Appliance Schedule ===")
for i, (name, _, _) in enumerate(appliances):
    print(f"{name:20s} → Start at hour {int(alpha[i])}")
print(f"Total cost for the day: ${alpha_score:.2f}")


Iteration  20 | Best cost so far: $3.96
Iteration  40 | Best cost so far: $3.91
Iteration  60 | Best cost so far: $3.91
Iteration  80 | Best cost so far: $3.84
Iteration 100 | Best cost so far: $3.84

=== Optimal Appliance Schedule ===
Washing Machine      → Start at hour 14
Dishwasher           → Start at hour 4
Heater               → Start at hour 13
Air Conditioner      → Start at hour 13
Total cost for the day: $3.84
